# Aplicaciones de Transformers en Procesamiento de Lenguaje Natural

Este cuaderno explora distintas aplicaciones de los modelos Transformers en tareas de procesamiento de lenguaje natural (PLN), utilizando ejemplos prácticos y explicaciones teóricas.

---

## ¿Qué es la arquitectura Transformer?

La **arquitectura Transformer** es un tipo de red neuronal propuesta por Vaswani et al. en 2017. Su principal innovación es el mecanismo de *atención*, que permite al modelo enfocarse en diferentes partes de la entrada para procesar información contextual de manera eficiente. Los Transformers reemplazaron a modelos anteriores como LSTM y GRU en muchas tareas de PLN debido a su capacidad para manejar secuencias largas y aprender dependencias complejas.

---

## ¿Qué es la librería Transformers de Hugging Face?

La **librería Transformers de Hugging Face** es una biblioteca de Python que proporciona implementaciones preentrenadas de modelos basados en la arquitectura Transformer (como BERT, GPT, T5, etc.) para tareas de PLN. Permite cargar modelos fácilmente y utilizarlos en tareas como clasificación, traducción, resumen, etc., sin necesidad de entrenarlos desde cero. Podes explorar los modelos disponibles en la [página de modelos de Hugging Face](https://huggingface.co/models).

---

## ¿Qué es un pipeline en Hugging Face?

Un **pipeline** en Hugging Face es una interfaz de alto nivel que simplifica el uso de modelos preentrenados para tareas específicas. Permite ejecutar tareas como clasificación de texto, reconocimiento de entidades, traducción, etc., con solo unas pocas líneas de código, ocultando detalles complejos de preprocesamiento y postprocesamiento.


## 1. Definir el texto de ejemplo

A continuación, se presenta un texto de ejemplo que será utilizado en las distintas aplicaciones de modelos Transformers. Este texto simula una queja de un cliente sobre un pedido equivocado, lo que nos permitirá explorar tareas como clasificación, reconocimiento de entidades, respuesta a preguntas, resumen, traducción y generación de texto.

In [ ]:
texto = """Querido MercadoLibre, la semana pasada pedí una figura de acción de Optimus Prime desde su tienda online.
Para mi sorpresa, cuando abrí el paquete, descubrí horrorizado que me habían enviado una figura de Megatron.
Como fan de los Autobots, espero que entiendan mi decepción. Solicito un cambio urgente del producto."""

## 2. Clasificación de texto con Transformers

La **clasificación de texto** es una tarea fundamental en PLN que consiste en asignar una o varias etiquetas a un texto, como por ejemplo identificar el sentimiento (positivo, negativo, neutro) o la intención del mensaje. Los modelos Transformers, gracias a su capacidad para comprender el contexto, demostraron un rendimiento sobresaliente en esta tarea.

Utilizaremos el pipeline `text-classification` de la librería 🤗 Transformers para analizar el texto de ejemplo y visualizar el resultado de la clasificación.

In [2]:
from transformers import pipeline
# Modelo de clasificación de texto (sentimiento) en español
# Puedes buscar otros modelos en https://huggingface.co/models
classifier = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis")
import pandas as pd
outputs = classifier(texto)
pd.DataFrame(outputs)

KeyboardInterrupt: 

## 3. Reconocimiento de entidades nombradas (NER)

El **Reconocimiento de Entidades Nombradas** (NER, por sus siglas en inglés) es la tarea de identificar y clasificar automáticamente nombres propios, lugares, organizaciones y otras entidades relevantes dentro de un texto. Los Transformers permiten realizar NER de manera eficiente y precisa.

A continuación, aplicamos el pipeline `ner` al texto de ejemplo para extraer las entidades nombradas presentes.

In [ ]:
# Modelo de reconocimiento de entidades nombradas (NER) en español
# Puedes buscar otros modelos en https://huggingface.co/models
ner_tagger = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

## 4. Respuesta a preguntas basada en contexto

La tarea de **respuesta a preguntas** (Question Answering) consiste en responder preguntas específicas utilizando un contexto dado. Los modelos Transformers pueden comprender el texto y extraer la información relevante para responder de manera precisa.

En el siguiente ejemplo, preguntamos al modelo qué desea el cliente, utilizando el pipeline `question-answering`.

In [ ]:
# Modelo de respuesta a preguntas en español
# Puedes buscar otros modelos en https://huggingface.co/models
reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")
question = "¿Qué quiere el cliente?"
outputs = reader(question=question, context=texto)
pd.DataFrame([outputs])

## 5. Resumen automático de texto

El **resumen automático** permite condensar la información principal de un texto extenso en una versión más corta, manteniendo el significado esencial. Los Transformers han revolucionado esta tarea gracias a su capacidad de comprensión contextual.

A continuación, utilizamos el pipeline `summarization` para obtener un resumen del texto de ejemplo.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# Puedes buscar otros modelos en https://huggingface.co/models
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
resumidor = pipeline("summarization", model=model, tokenizer=tokenizer)
resumen = resumidor(texto, max_length=80, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])

## 6. Traducción automática del español a inglés

La **traducción automática** es una de las aplicaciones más destacadas de los Transformers, permitiendo traducir textos entre diferentes idiomas con alta calidad. Utilizaremos el pipeline `translation_es_to_en` para traducir el texto de ejemplo del español al inglés.

In [ ]:
# Modelo de traducción de español a inglés
# Puedes buscar otros modelos en https://huggingface.co/models
translator = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")
outputs = translator(texto)
print(outputs[0]['translation_text'])

## 7. Generación automática de texto (respuesta de servicio al cliente)

La **generación automática de texto** permite crear o continuar textos de manera coherente a partir de un prompt inicial. Esta capacidad es útil, por ejemplo, para redactar respuestas automáticas en atención al cliente.

En el siguiente ejemplo, generamos una posible respuesta de servicio al cliente utilizando el pipeline `text-generation`.

In [ ]:
# Modelo de generación de texto en español
# Puedes buscar otros modelos en https://huggingface.co/models
generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "
prompt = texto + "\n\nRespuesta del servicio al cliente:\n" + respuesta_inicial
outputs = generator(
    prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=50256
)
print(outputs[0]['generated_text'])

---

## Ejercicios autónomos (40 minutos)

A continuación, te proponemos ejercicios prácticos para que pongas en práctica lo aprendido sobre la arquitectura Transformer, la librería Transformers de Hugging Face y el uso de pipelines en tareas de PLN.

**Recorda:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y busca modelos que sean apropiados para español, o incluso para el contexto argentino si los encontras.

### 1. Clasificación de textos propios
- Escribi dos textos breves (pueden ser opiniones, quejas o comentarios sobre servicios o productos en Argentina, por ejemplo sobre colectivos, supermercados, bancos, etc.).
- Busca en Hugging Face un modelo de clasificación de texto en español y usalo con el pipeline `text-classification` para analizar el sentimiento de cada texto.
- ¿El resultado coincide con tu expectativa? Explica brevemente.

### 2. Reconocimiento de entidades en noticias argentinas
- Busca un párrafo de una noticia reciente de un medio argentino (por ejemplo, Clarín, La Nación, Página/12).
- Busca en Hugging Face un modelo de NER para español y aplícalo para identificar entidades nombradas.
- Enumera las entidades encontradas y clasifícalas (persona, organización, lugar, etc.).

### 3. Respuesta a preguntas personalizada
- Escribi un pequeño texto narrativo (4-5 líneas) sobre una situación cotidiana en Buenos Aires (por ejemplo, un reclamo en una oficina pública, una experiencia en el subte, etc.).
- Formula dos preguntas sobre el texto y utiliza un modelo de `question-answering` de Hugging Face para responderlas.
- ¿Las respuestas son correctas y precisas? ¿Por qué?

### 4. Resumen de un texto propio
- Elegí un texto más largo (puede ser un fragmento de Wikipedia sobre Argentina, una noticia, etc.).
- Busca un modelo de resumen en español en Hugging Face y úsalo con el pipeline `summarization`.
- Compara el resumen generado con el texto original: ¿qué información se perdió y cuál se mantuvo?

### 5. Traducción y análisis
- Escribi un texto breve en español rioplatense y tradúcilo al inglés usando un modelo de traducción de Hugging Face.
- Luego, traduci el resultado nuevamente al español (podes usar otro pipeline o Google Translate).
- ¿El texto final es igual al original? ¿Qué diferencias encontras?

### 6. Generación creativa de texto
- Escribe el inicio de una historia o una pregunta abierta relacionada con Buenos Aires.
- Busca un modelo de generación de texto en español en Hugging Face y úsalo con el pipeline `text-generation` para continuar el texto.
- Analiza la coherencia y creatividad de la respuesta generada.

---

**Tip:** Para cada ejercicio, explora la [página de modelos de Hugging Face](https://huggingface.co/models) y lee la descripción de los modelos antes de usarlos. Si tienes dudas, consulta la documentación oficial de [Transformers](https://huggingface.co/docs/transformers/index).

## 1

In [4]:
texto1= "Que frio que va a hacer en Capital esta semana."
texto2 = "La guitarra sonó muy bien en el concierto."

In [5]:
from transformers import pipeline
# Modelo de clasificación de texto (sentimiento) en español
# Puedes buscar otros modelos en https://huggingface.co/models
classifier = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")
import pandas as pd
output1 = classifier(texto1)

pd.DataFrame(output1)

Device set to use cpu


,label,score
0,Very Negative,0.644534


In [ ]:
output2 = classifier(texto2)

pd.DataFrame(output2)

,label,score
0,Positive,0.550817


## 2

In [6]:
texto = "Estos agentes están empezando a reemplazar roles en programación, atención al cliente y análisis financiero. Mark Zuckerberg, CEO de Meta, ha pronosticado que para 2025, dichos agentes de IA ya estarán desempeñando funciones de ingenieros de nivel medio, lo que reducirá la necesidad de trabajadores humanos en áreas críticas."

In [ ]:
ner_tagger = pipeline("ner", model="MMG/xlm-roberta-large-ner-spanish", aggregation_strategy="simple")
outputs = ner_tagger(texto)
pd.DataFrame(outputs)

Device set to use cpu


,entity_group,score,word,start,end
0,PER,0.998255,Mark Zuckerberg,109,124
1,ORG,0.586556,CEO,126,129
2,ORG,0.971456,Meta,133,137
3,MISC,0.828076,IA,188,190


# 3

In [7]:
texto = """
Hoy en el subte de la línea B, en la estación Pueyrredón, un grupo de jóvenes con bombos y banderas empezó a tocar y cantar cumbia.
La gente, sorprendida al principio, terminó aplaudiendo y algunos hasta bailaron en el vagón.
Fue un momento inesperado y alegre en medio de la rutina de la mañana.
"""

preguntas = [
    "¿Qué ocurrió en la estación Pueyrredón?",
    "¿Qué tipo de música tocaron los jóvenes?"
]

reader = pipeline("question-answering", model="PlanTL-GOB-ES/roberta-large-bne-sqac")

for pregunta in preguntas:
    outputs = reader(question=pregunta, context=texto)
    print(f"Pregunta: {pregunta}")
    print(f"Respuesta: {outputs['answer']}")
    print("-" * 20)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


Pregunta: ¿Qué ocurrió en la estación Pueyrredón?
Respuesta: empezó a tocar y cantar cumbia
--------------------
Pregunta: ¿Qué tipo de música tocaron los jóvenes?
Respuesta: cumbia
--------------------


# 4

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

texto_largo = """
El retrato de Lisa Gherardini, esposa de Francesco del Giocondo,[1]​ más conocido como La Gioconda (La Joconde en francés) o Monna Lisa, es una obra pictórica del polímata renacentista florentino Leonardo da Vinci. Fue adquirida por el rey Francisco I de Francia a comienzos del siglo XVI y desde entonces es propiedad del Estado francés. Se halla expuesta en el Museo del Louvre de París, siendo, sin duda, la «joya» de sus colecciones.

Gioconda significa "esposa alegre o jovial" en italiano. Hay 12 santorales con ese nombre, unos masculinos, otros femeninos. Una tésis del nombre del óleo, La Gioconda, la más aceptada es sobre la identidad de la modelo: la esposa de Francesco Bartolomeo de Giocondo, que realmente se llamaba Lisa Gherardini, de donde viene su otro nombre: Monna (señora, en el italiano antiguo) Lisa. El Museo del Louvre acepta el título completo indicado al principio como el título original de la obra, aunque no reconoce la identidad de la modelo y tan solo la acepta como una hipótesis.[2]​
"""

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Generate the summary
# Adjust max_length and min_length as needed to control the summary length
summary = summarizer(texto_largo, max_length=100, min_length=30, do_sample=False)

print("Original:")
print(texto_largo)
print("\nResumen:")
print(summary[0]['summary_text'])

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:  10%|#         | 273M/2.60G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Device set to use cpu


Original:

El retrato de Lisa Gherardini, esposa de Francesco del Giocondo,[1]​ más conocido como La Gioconda (La Joconde en francés) o Monna Lisa, es una obra pictórica del polímata renacentista florentino Leonardo da Vinci. Fue adquirida por el rey Francisco I de Francia a comienzos del siglo XVI y desde entonces es propiedad del Estado francés. Se halla expuesta en el Museo del Louvre de París, siendo, sin duda, la «joya» de sus colecciones.

Gioconda significa "esposa alegre o jovial" en italiano. Hay 12 santorales con ese nombre, unos masculinos, otros femeninos. Una tésis del nombre del óleo, La Gioconda, la más aceptada es sobre la identidad de la modelo: la esposa de Francesco Bartolomeo de Giocondo, que realmente se llamaba Lisa Gherardini, de donde viene su otro nombre: Monna (señora, en el italiano antiguo) Lisa. El Museo del Louvre acepta el título completo indicado al principio como el título original de la obra, aunque no reconoce la identidad de la modelo y tan solo la a

# 5

In [12]:
texto = "Che, ¿todo bien? Acá en Buenos Aires está re fresco."

traductor_es_en = pipeline("translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")

traduccion_eng = traductor_es_en(texto)
eng_texto = traduccion_eng[0]['translation_text']
print(f"Original: {texto}")
print(f"Traducido a ingles: {eng_texto}")

traductor_en_es = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")

traduccion_es = traductor_en_es(eng_texto)
es_texto = traduccion_es[0]['translation_text']
print(f"Traducido a español: {es_texto}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Traducido a español: Aquí en Buenos Aires es fresco.


# 6

In [13]:
from transformers import pipeline

prompt = "En una noche de lluvia en Buenos Aires, el tango sonaba en un viejo bar de San Telmo..."

generator = pipeline("text-generation", model="datificate/gpt2-small-spanish")

outputs = generator(
    prompt,
    max_new_tokens=100,  # Adjust the number of new tokens to generate
    do_sample=True,      # Set to True for more creative and diverse output
    temperature=0.7,     # Controls the randomness of the output (lower = more predictable)
    top_k=50,            # Limits the sampling to the top_k most likely tokens
    top_p=0.9,           # Limits the sampling to the smallest set of tokens whose cumulative probability exceeds top_p
    repetition_penalty=1.2 # Penalizes tokens that have already appeared
)

print(outputs[0]['generated_text'])

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Device set to use cpu


En una noche de lluvia en Buenos Aires, el tango sonaba en un viejo bar de San Telmo... La ciudad estaba totalmente cubierta por la luz del sol.

Los músicos no tuvieron que luchar para encontrar algo más grande y pesado con el tango a principios de los años 1930: Los guitarristas Julio Iglesias (guitarra), Jorge Alende (batería) y Julio Sosa (bajo). En esta época, el tango era interpretado regularmente como una forma musical alternativa al tango clásico o folclórica argentino; por ejemplo, en "La marcha" (1937-39), el músico Héctor «Pibe» Castro cantó junto a
